### 資料集簡介
<p>欄位說明:</p>
<p>FileID: 檔案識別ID</p>
<p>CustomerID: 使用者裝置識別ID</p>
<p>QueryTs: 該筆資料發生時間</p>
<p>ProductID: 使用者裝置的產品代碼</p>



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family']='SimHei' #顯示中文

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load in the train datasets
train = pd.read_csv('input/training-set.csv', encoding = "utf-8", header=None)
test = pd.read_csv('input/testing-set.csv', encoding = "utf-8", header=None)

In [4]:
#query_log裡面被官方排除的 FileID
train_exc = pd.read_csv('input/exception/exception_train.txt', encoding = "utf-8", header=None)
test_exc = pd.read_csv('input/exception/exception_testing.txt', encoding = "utf-8", header=None)

In [5]:
test_exc.head(2)

,0
0,4eedf630f7160dafea969e5d57239d31
1,8e1c12f42b705cb465bab88225d03c81


In [6]:
# from sklearn.preprocessing import Imputer
# imputer = Imputer(missing_values='NaN', axis=0, strategy='mean') 
# imputer.fit_transform(X[:,[1,3]])

In [7]:
# training set - label: 0:非惡意程式, 1:惡意程式
train.columns=['FileID','label']
train.head(2)

,FileID,label
0,0000e2398b12121a85166fed5fe2a3da,0
1,0001fe8dce14ce099aa6ca8ea5026ea7,0


In [8]:
# testing set - AUC: Area Under ROC Curve
test.columns=['FileID','AUC']
test.head(2)

,FileID,AUC
0,00008c73ee43c15b16c26b26398c1577,0.5
1,0002ded3a0b54f2ffdab0ca77a5ce2b6,0.5


In [9]:
#確認排除的FileID在training set裡面找不到
train[train['FileID'].isin(train_exc[0])]

,FileID,label


In [10]:
#取0301當天的query log來查看
query_0301 = pd.read_csv('input/query_log/0301.csv', encoding = "utf-8", header=None)
query_0301.columns=['FileID','CustomerID','QueryTs','ProductID']
query_0301['times'] = 1

In [11]:
query_0301.head(2)

,FileID,CustomerID,QueryTs,ProductID,times
0,dfccd8e23f0b03ec4db7a9a745ad7399,64573086fcfee53e8fa6d8f1f44101bb,1488326402,c105a0,1
1,29962f9102502ee76bb3cddb8cf8c9ad,597df153a25e8e548cbe35f6e4da9cbb,1488326403,c105a0,1


In [12]:
query_0301.describe()

,QueryTs,times
count,4.755690e+05,475569.0
mean,1.488366e+09,1.0
std,2.451022e+04,0.0
min,1.488326e+09,1.0
25%,1.488346e+09,1.0
50%,1.488365e+09,1.0
75%,1.488385e+09,1.0
max,1.488413e+09,1.0


In [13]:
query_0301.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475569 entries, 0 to 475568
Data columns (total 5 columns):
FileID        475569 non-null object
CustomerID    475569 non-null object
QueryTs       475569 non-null int64
ProductID     475569 non-null object
times         475569 non-null int64
dtypes: int64(2), object(3)
memory usage: 116.6 MB


### 樞杻分析

In [14]:
query_0301.pivot_table(values='times',index=['FileID'],columns='ProductID',aggfunc='sum')

ProductID,055649,20f8a5,262880,26a5d0,3ea8c3,533133,634e6b,7acab3,8541a0,885fab,c105a0,c76d58,d465fc,e47f04
FileID,,,,,,,,,,,,,,
0087304d5bee451f0d82c7407b0fa7d1,NaN,8.0,NaN,NaN,7.0,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN
0087e85caf760ff171b1774bafcafcae,45.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,3.0,NaN,NaN
00a0fadf6f4b4d244459b3bc8f6ee7fa,NaN,NaN,NaN,NaN,52.0,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00a885e87beb6fb1f6ca6ee9eb783779,8.0,NaN,NaN,NaN,2.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,1.0
00a92b584bfcf58b70d5e965e91da03c,28.0,30.0,NaN,NaN,1.0,NaN,16.0,53.0,NaN,NaN,3.0,1.0,NaN,NaN
00cc3e15a0f2aad83d53752c10fbd2e3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN
00d37d5360716660f7f51b14053d4c8d,12.0,NaN,NaN,NaN,NaN,NaN,NaN,56.0,NaN,NaN,NaN,1.0,NaN,NaN
00ff41de2fab610efb3a2a0a1e6a525e,11.0,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,1.0,NaN,NaN
011369beb2e171a204a2d7ffe86fcda9,56.0,NaN,NaN,NaN,NaN,NaN,NaN,194.0,NaN,NaN,NaN,7.0,NaN,NaN


### 聚合函數
count（個數）, sum（加總）, mean（平均）, median（中位數）, std（標準差）, var（變異數）, first（第一個非NA）, last（最後一個非NA）

In [15]:
query_0301.groupby(['FileID','CustomerID','ProductID'])[['times']].sum()

times
FileID                           CustomerID                       ProductID       
0087304d5bee451f0d82c7407b0fa7d1 282396145a3df4452761bacf8049f6db 3ea8c3         7
                                 501782bb051c5368f641f5299e550f32 20f8a5         8
                                 54a57555a85251b45a9de37ba95a4cf3 7acab3         2
                                 662a1f7c7441bace48defd0f1947bc36 7acab3         2
                                 71a15fcfe2741879c0d3a4588c011dae 7acab3         2
                                 bb0b6c51b1fd781a7a4cd1b2365ba889 7acab3         2
                                 d61e6c11ba5b56de990e512e86b51886 7acab3         2
0087e85caf760ff171b1774bafcafcae 002a0a48cb30742c0a85c8661384edb9 055649         1
                                 01d32fea310f8b1cd8a98dd57745918b 7acab3         1
                                 02feea730f4de6baf2e0675835e94623 055649         1
                                 0456e70e10362f10f21ab8629c0704f8 7acab3         1
                                 0693bb9028788dd05e691d5b41903a2a 7acab3         1
                                 1160ca9bd5b424af814971c1897c3dbc c76d58         1
                                 12b17df2178e0bbd99fa144903fe0afb 7acab3         1
                                 144ca08c86870967bde66396ba32669f 055649         1
                                 1638281433537f34fb53eb6e0a5d6bb6 7acab3         1
                                 1a248fe68eb19b599d3a03d2936e388e 7acab3         1
                                 1ca40a65ec6f71efe2b4f840b859b4bf 055649         1
                                 1e180bb56546440e7fa7738af0441c5c 055649         1
                                 216e2ac1cb62a0e39ec8f779130f3255 055649         1
                                 25b402190079b38b5c7087c7007f7238 7acab3         1
                                 30e96e9a8d6036a799210b9a5a5232e3 055649         1
                                 34144d1fac98512a2733b3683956cdb5 055649         1
                                 37045bfc8dd54190fcbb01ab8aecab10 055649         1
                                 3811693a78ffabbfc5bb0b4adb526104 7acab3         1
                                 3b26072d43bed713e4f01ba2378daf7a 055649         1
                                 40bf9554af692dc7682a70627747a0ef 055649         1
                                 45def3125f5bca7f5047931d5a738aa8 7acab3         1
                                 466bc59569344ceeaa8d25cd73a041ce 7acab3         1
                                 488b40fb2bb5b52ae92f4afb5854cc48 055649         1
...                                                                            ...
ffe023042f06cb48603058b6245fc338 75ce7c4172a145839f58642dc952668a c105a0         1
                                 76699e4bcfbde761d45fc503c92ced50 c105a0         1
                                 7823248eea1343f107b3aaa712614be4 c105a0         3
                                 7a83fb37d4c12973be776212528cb31d c105a0         5
                                 7cd9c71fe0a56aa2c88d21f02fe2ccef c105a0         1
                                 7ea4fa751875391b73293c21c14aa432 055649         1
                                 7fec86081b1d691268e4393cc8e9cee0 7acab3         2
                                 8782a2c4039d65a9434c9ffbe77198e0 c105a0         1
                                 8ce468a7cf0a985ef275aad76d85a8d6 c105a0         3
                                 8e9ae304354117f5094c618a78f0df02 c105a0         1
                                 98b4e58ea0d7c7a5d98ebb6f9edd234b c105a0         2
                                 9a3ee4eda7e837d924af9c7fb5b3aeff c105a0         1
                                 9a40017505bdc8522055268152de455e c105a0         3
                                 9e786ac444ec164122bad895d4cf07eb 055649         1
                                 a19bd4ac97e19ad42f642b36e394858a 055649         2
                                 ad894d213675d725e1a36eadc10bdf63 c105a0         3
          

### 其它會用到的工具

In [16]:
#AUC計算範例
import numpy as np
from sklearn import metrics
y = np.array([1, 1, 2, 2])
pred = np.array([0.5, 1, 0.9, 1])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
metrics.auc(fpr, tpr)


0.625

In [17]:
#timestamp轉換
import datetime
print(
    datetime.datetime.fromtimestamp(
        int("1488326402")
    ).strftime('%Y-%m-%d %H:%M:%S'))


2017-03-01 08:00:02
